In [22]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 16: Reindeer Maze ---</h2><p>It's time again for the <a href="/2015/day/14">Reindeer Olympics</a>! This year, the big event is the <em>Reindeer Maze</em>, where the Reindeer compete for the <em><span title="I would say it's like Reindeer Golf, but knowing Reindeer, it's almost certainly nothing like Reindeer Golf.">lowest score</span></em>.</p>
<p>You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?</p>
<p>The Reindeer start on the Start Tile (marked <code>S</code>) facing <em>East</em> and need to reach the End Tile (marked <code>E</code>). They can move forward one tile at a time (increasing their score by <code>1</code> point), but never into a wall (<code>#</code>). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by <code>1000</code> points).</p>
<p>To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:</p>
<pre><code>###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
</code></pre>
<p>There are many paths through this maze, but taking any of the best paths would incur a score of only <code><em>7036</em></code>. This can be achieved by taking a total of <code>36</code> steps forward and turning 90 degrees a total of <code>7</code> times:</p>
<pre><code>
###############
#.......#....<em>E</em>#
#.#.###.#.###<em>^</em>#
#.....#.#...#<em>^</em>#
#.###.#####.#<em>^</em>#
#.#.#.......#<em>^</em>#
#.#.#####.###<em>^</em>#
#..<em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>v</em>#<em>^</em>#
###<em>^</em>#.#####<em>v</em>#<em>^</em>#
#<em>&gt;</em><em>&gt;</em><em>^</em>#.....#<em>v</em>#<em>^</em>#
#<em>^</em>#.#.###.#<em>v</em>#<em>^</em>#
#<em>^</em>....#...#<em>v</em>#<em>^</em>#
#<em>^</em>###.#.#.#<em>v</em>#<em>^</em>#
#S..#.....#<em>&gt;</em><em>&gt;</em><em>^</em>#
###############
</code></pre>
<p>Here's a second example:</p>
<pre><code>#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
</code></pre>
<p>In this maze, the best paths cost <code><em>11048</em></code> points; following one such path would look like this:</p>
<pre><code>#################
#...#...#...#..<em>E</em>#
#.#.#.#.#.#.#.#<em>^</em>#
#.#.#.#...#...#<em>^</em>#
#.#.#.#.###.#.#<em>^</em>#
#<em>&gt;</em><em>&gt;</em><em>v</em>#.#.#.....#<em>^</em>#
#<em>^</em>#<em>v</em>#.#.#.#####<em>^</em>#
#<em>^</em>#<em>v</em>..#.#.#<em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>^</em>#
#<em>^</em>#<em>v</em>#####.#<em>^</em>###.#
#<em>^</em>#<em>v</em>#..<em>&gt;</em><em>&gt;</em><em>&gt;</em><em>&gt;</em><em>^</em>#...#
#<em>^</em>#<em>v</em>###<em>^</em>#####.###
#<em>^</em>#<em>v</em>#<em>&gt;</em><em>&gt;</em><em>^</em>#.....#.#
#<em>^</em>#<em>v</em>#<em>^</em>#####.###.#
#<em>^</em>#<em>v</em>#<em>^</em>........#.#
#<em>^</em>#<em>v</em>#<em>^</em>#########.#
#S#<em>&gt;</em><em>&gt;</em><em>^</em>..........#
#################
</code></pre>
<p>Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.</p>
<p>Analyze your map carefully. <em>What is the lowest score a Reindeer could possibly get?</em></p>
</article>


In [23]:
from heapq import heappop, heappush
from more_itertools import first


tests = [
    {
        "name": "Example 1",
        "s": """
            ###############
            #.......#....E#
            #.#.###.#.###.#
            #.....#.#...#.#
            #.###.#####.#.#
            #.#.#.......#.#
            #.#.#####.###.#
            #...........#.#
            ###.#.#####.#.#
            #...#.....#.#.#
            #.#.#.###.#.#.#
            #.....#...#.#.#
            #.###.#.#.#.#.#
            #S..#.....#...#
            ###############
        """,
        "expected": 7036,
    },
    {
        "name": "Example 2",
        "s": """
            #################
            #...#...#...#..E#
            #.#.#.#.#.#.#.#.#
            #.#.#.#...#...#.#
            #.#.#.#.###.#.#.#
            #...#.#.#.....#.#
            #.#.#.#.#.#####.#
            #.#...#.#.#.....#
            #.#.#####.#.###.#
            #.#.#.......#...#
            #.#.###.#####.###
            #.#.#...#.....#.#
            #.#.#.#####.###.#
            #.#.#.........#.#
            #.#.#.#########.#
            #S#.............#
            #################
        """,
        "expected": 11048,
    },
]


class Maze(Str):
    def __init__(self, s: str) -> None:
        self.maze = [list(l.strip()) for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.maze), len(self.maze[0])
        self.S = self._get(key="S")
        self.E = self._get(key="E")

    def lowest_score(self) -> int:
        heap = [(0, *self.S, 0, 1)]
        seen = set()

        while heap:
            score, r, c, dr, dc = heappop(heap)

            if (r, c) == self.E:
                return score

            if (r, c, dr, dc) in seen:
                continue

            seen.add((r, c, dr, dc))

            if self.maze[r + dr][c + dc] != "#":
                heappush(heap, (score + 1, r + dr, c + dc, dr, dc))

            heappush(heap, (score + 1000, r, c, *self.clockwise(dr, dc)))
            heappush(heap, (score + 1000, r, c, *self.counter_clockwise(dr, dc)))

        return -1

    @staticmethod
    def clockwise(dr: int, dc: int) -> tuple[int, int]:
        return -dc, dr

    @staticmethod
    def counter_clockwise(dr: int, dc: int) -> tuple[int, int]:
        return dc, -dr

    def _get(self, key) -> tuple[int, int]:
        return first(
            (r, c)
            for r, c in product(range(self.rows), range(self.cols))
            if self.maze[r][c] == key
        )

    def __str__(self) -> str:
        return "\n".join("".join(l) for l in self.maze)


@test(tests=tests[:])
def partI_test(s: str) -> int:
    m = Maze(s)
    return m.lowest_score()


Test Example 1 passed, for partI_test.
Test Example 2 passed, for partI_test.
Success


In [24]:
with open("../input/day16.txt") as f:
    puzzle = f.read()

print(f"Part I: {Maze(puzzle).lowest_score()}")

Part I: 130536


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>130536</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you know what the best paths look like, you can figure out the best spot to sit.</p>
<p>Every non-wall tile (<code>S</code>, <code>.</code>, or <code>E</code>) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is <em>whether the tile is on one of the best paths through the maze</em>. If you sit somewhere else, you'd miss all the action!</p>
<p>So, you'll need to determine which tiles are part of <em>any</em> best path through the maze, including the <code>S</code> and <code>E</code> tiles.</p>
<p>In the first example, there are <code><em>45</em></code> tiles (marked <code>O</code>) that are part of at least one of the various best paths through the maze:</p>
<pre><code>###############
#.......#....<em>O</em>#
#.#.###.#.###<em>O</em>#
#.....#.#...#<em>O</em>#
#.###.#####.#<em>O</em>#
#.#.#.......#<em>O</em>#
#.#.#####.###<em>O</em>#
#..<em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em>#<em>O</em>#
###<em>O</em>#<em>O</em>#####<em>O</em>#<em>O</em>#
#<em>O</em><em>O</em><em>O</em>#<em>O</em>....#<em>O</em>#<em>O</em>#
#<em>O</em>#<em>O</em>#<em>O</em>###.#<em>O</em>#<em>O</em>#
#<em>O</em><em>O</em><em>O</em><em>O</em><em>O</em>#...#<em>O</em>#<em>O</em>#
#<em>O</em>###.#.#.#<em>O</em>#<em>O</em>#
#<em>O</em>..#.....#<em>O</em><em>O</em><em>O</em>#
###############
</code></pre>
<p>In the second example, there are <code><em>64</em></code> tiles that are part of at least one of the best paths:</p>
<pre><code>#################
#...#...#...#..<em>O</em>#
#.#.#.#.#.#.#.#<em>O</em>#
#.#.#.#...#...#<em>O</em>#
#.#.#.#.###.#.#<em>O</em>#
#<em>O</em><em>O</em><em>O</em>#.#.#.....#<em>O</em>#
#<em>O</em>#<em>O</em>#.#.#.#####<em>O</em>#
#<em>O</em>#<em>O</em>..#.#.#<em>O</em><em>O</em><em>O</em><em>O</em><em>O</em>#
#<em>O</em>#<em>O</em>#####.#<em>O</em>###<em>O</em>#
#<em>O</em>#<em>O</em>#..<em>O</em><em>O</em><em>O</em><em>O</em><em>O</em>#<em>O</em><em>O</em><em>O</em>#
#<em>O</em>#<em>O</em>###<em>O</em>#####<em>O</em>###
#<em>O</em>#<em>O</em>#<em>O</em><em>O</em><em>O</em>#..<em>O</em><em>O</em><em>O</em>#.#
#<em>O</em>#<em>O</em>#<em>O</em>#####<em>O</em>###.#
#<em>O</em>#<em>O</em>#<em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em><em>O</em>..#.#
#<em>O</em>#<em>O</em>#<em>O</em>#########.#
#<em>O</em>#<em>O</em><em>O</em><em>O</em>..........#
#################
</code></pre>
<p>Analyze your map further. <em>How many tiles are part of at least one of the best paths through the maze?</em></p>
</article>


In [25]:
from collections import deque
from heapq import heappop, heappush
from more_itertools import collapse, first


tests = [
    {
        "name": "Example 1",
        "s": """
            ###############
            #.......#....E#
            #.#.###.#.###.#
            #.....#.#...#.#
            #.###.#####.#.#
            #.#.#.......#.#
            #.#.#####.###.#
            #...........#.#
            ###.#.#####.#.#
            #...#.....#.#.#
            #.#.#.###.#.#.#
            #.....#...#.#.#
            #.###.#.#.#.#.#
            #S..#.....#...#
            ###############
        """,
        "expected": 45,
    },
    {
        "name": "Example 2",
        "s": """
            #################
            #...#...#...#..E#
            #.#.#.#.#.#.#.#.#
            #.#.#.#...#...#.#
            #.#.#.#.###.#.#.#
            #...#.#.#.....#.#
            #.#.#.#.#.#####.#
            #.#...#.#.#.....#
            #.#.#####.#.###.#
            #.#.#.......#...#
            #.#.###.#####.###
            #.#.#...#.....#.#
            #.#.#.#####.###.#
            #.#.#.........#.#
            #.#.#.#########.#
            #S#.............#
            #################
        """,
        "expected": 64,
    },
]


class MazeII(Maze):
    def tiles_smallest_paths(self, debug=False) -> int:
        queue = deque([(0, *self.S, 0, 1, [])])

        min_score = inf
        min_score_paths = []
        min_scores = {}

        while queue:
            score, r, c, dr, dc, path = queue.popleft()

            if score > min_score:
                continue

            if score > min_scores.get((r, c, dr, dc), inf):
                continue

            min_scores[(r, c, dr, dc)] = score

            path = path + [(r, c)]

            if (r, c) == self.E:
                if score < min_score:
                    min_score = score
                    min_score_paths = []

                min_score_paths.append(path)
                continue

            if self.maze[r + dr][c + dc] != "#":
                queue.append((score + 1, r + dr, c + dc, dr, dc, path))

            drr, dcc = self.clockwise(dr, dc)
            if self.maze[r + drr][c + dcc] != "#":
                queue.append((score + 1001, r, c, drr, dcc, path))

            drr, dcc = self.counter_clockwise(dr, dc)
            if self.maze[r + drr][c + dcc] != "#":
                queue.append((score + 1001, r, c, drr, dcc, path))

        if debug:
            print(self.__str__(min_score_paths))
            print(f"{set(collapse(min_score_paths))}")
        return len(set(collapse(min_score_paths, levels=1)))

    def __str__(self, tiles: set[tuple[int, int]] = set()) -> str:
        return "\n".join(
            "".join(
                "O" if (r, c) in tiles else self.maze[r][c] for c in range(self.cols)
            )
            for r in range(self.rows)
        )


@test(tests=tests[:])
def partI_test(s: str) -> int:
    m = MazeII(s)
    return m.tiles_smallest_paths()


Test Example 1 passed, for partI_test.
Test Example 2 passed, for partI_test.
Success


In [26]:
print(f"Part II: { MazeII(puzzle).tiles_smallest_paths() }")

Part II: 1024


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1024 </code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
